In [3]:
import gym
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

from collections import deque
import gym
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from gym import wrappers
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.optimizers import SGD, RMSprop, Adam, Adamax

Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
env = gym.make("LunarLander-v2")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [ ]:
GAMMA = 0.99
EPSILON_START = 1
EPSILON_END = .005
EPSILON_DECAY = .995
LR = 5e-4
UPDATE_EVERY = 4
REPLACE_EVERY = 500
TAU = .001
BATCH_SIZE = 64
BUFFER_SIZE = 10000

class DQN():
    def __init__(self, env, eps):
        # Tell the agent what game space we're playing in
        self.state_size = env.observation_space.shape[0]
        self.action_size = env.action_space.n
        self.env = env
                
        # Create any parameters we'll be adjusting internally
        self.epsilon = eps
        
        # Creating the neural networks
        self.qnetwork_expected = self.create_model()
        self.qnetwork_target = self.create_model()
        
        # Creating the memory for experience replay, and a time step
        self.memory = deque(maxlen=BUFFER_SIZE)
        self.t_step = 0
        self.t_step_2 = 0
        self.episodes = 0
        
    def create_model(self):
        model   = Sequential()
        model.add(Dense(64, activation="relu", input_shape=(8,)))
        model.add(Dense(64, activation="relu"))
        model.add(Dense(self.action_size))
        model.compile(loss="mean_squared_error",
            optimizer=Adam(lr=LR))
        return model
    
    def act(self, state):
        # Take a random action if we roll a number less than epsilon, else do what our train network says to do
        threshold = random.random()
        if threshold < self.epsilon:
            action = self.env.action_space.sample()
        if threshold >= self.epsilon:
            action = np.argmax(self.qnetwork_expected.predict(state)[0])

#         # Decay epsilon after each action
#         if self.epsilon > EPSILON_END:
#             self.epsilon *= EPSILON_DECAY
#             self.epsilon = max(EPSILON_END, self.epsilon)
        
        return action
    
    def step(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

        # Set how often we'll update our train network weights
        self.t_step += 1 
        if self.t_step % UPDATE_EVERY == 0:
            if len(self.memory) > BATCH_SIZE:
                self.update()

        # Set how often we'll update our target network with train weights
        self.t_step_2 += 1
        if self.t_step_2 % UPDATE_EVERY ==0:
            self.train_target()

        # Tell it to pop if the memory is at buffer size
        if len(self.memory) > BUFFER_SIZE:
            self.memory.popleft()
    
    def update(self):
        samples = random.sample(self.memory, BATCH_SIZE)
        for sample in samples:
            state, action, reward, next_state, done = sample
            target = self.qnetwork_target.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future = max(self.qnetwork_target.predict(next_state)[0])
                target[0][action] = reward + Q_future * GAMMA
            self.qnetwork_expected.fit(state, target, epochs=2, verbose=0)

    def train_target(self):
        weights = self.qnetwork_expected.get_weights()
        target_weights = self.qnetwork_target.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * TAU + target_weights[i] * (1 - TAU)
        self.qnetwork_target.set_weights(target_weights)

    def save_model(self):
        qnetwork_expected.save('weights.h5')

In [ ]:
def main():
    env = gym.make("LunarLander-v2")
    trials  = 1000
    trial_len = 500
    eps_decay = 0.995
    epsilon = 1
    dqn_agent = DQN(env=env, eps = epsilon)
    
    def play_one(env, epsilon):
        dqn_agent = DQN(env=env, eps = epsilon)
        state = env.reset()
        print(state)
        state = env.reset().reshape(1,8)
        done = False
        full_reward_received = False
        totalreward = 0
        steps = 0
        while not done:
            action = dqn_agent.act(state)
            new_state, reward, done, info = env.step(action)
            new_state = new_state.reshape(1,8)
            dqn_agent.step(state, action, reward, new_state, done)
            totalreward += reward
            steps += 1
        return totalreward, steps
    
    n = 10000
    totalrewards = np.empty(n)
    for i in range (n):
        totalreward, steps = play_one(env=env, epsilon = epsilon)
        totalrewards[i] = totalreward
        epsilon *= eps_decay
        if i % 5 == 0:
            print("episode:", i, "total reward:", totalreward, "epsilon:", epsilon, "avg reward (last 50):", totalrewards[max(0, i-50):(i+1)].mean())
        if totalrewards[max(0, n-100):(n+1)].mean() >= 200:
            dqn_agent.save_model 
            break
main()

In [126]:
epsilon = 1
dqn_agent = DQN(env=env, eps = epsilon)
dqn_agent.save_model 


<bound method DQN.save_model of <__main__.DQN object at 0x1a5e4e4128>>